In [21]:

#!pip install matplotlib --force-reinstall --user
import matplotlib
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

plt.style.use('ggplot')
from transformers import  BertTokenizer,DistilBertTokenizer, RobertaModel, RobertaTokenizer
from transformers import  AdamW, get_linear_schedule_with_warmup
import torch
from torch import nn
from sklearn.model_selection import train_test_split
import os
import sys






In [22]:
import sys
import os
sys.path.append(os.path.abspath("/BERTFamily.py"))
import BERTFamily as fn

global variables:

In [23]:

RANDOM_SEED = 177
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
MAX_LENGTH = 64
BATCH_SIZE = 16
NUM_CLASSES = 3 # neutral, positive, negative
EPOCHS = 5
DROPOUT_PROB = 0.1
WEIGHT_DECAY = 0.01
NFOLDS = 10
LEARNING_RATE = 2e-5
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device=torch.device('cpu')
loss_function = nn.CrossEntropyLoss().to(device)

Dataset:

model training dataset and testing dataset: all-data.csv

In [24]:
statement_df = pd.read_csv('./data NASDAQ/all-data.csv', encoding='latin-1',
                           header=None)
statement_df.columns = ['sentiment', 'statement']
statement_df = statement_df.drop_duplicates()
statement_df['statement'] = statement_df['statement'].apply(fn.clean_statements)
le = LabelEncoder()
statement_df['sentiment'] = le.fit_transform(statement_df['sentiment'])
#positive-2, neutral-1,negative-0
df_train, df_test = train_test_split(statement_df,
                                     test_size=0.2,
                                     random_state=RANDOM_SEED,
                                     stratify=statement_df['sentiment'].values)

df_val, df_test = train_test_split(df_test,
                                   test_size=0.5,
                                   random_state=RANDOM_SEED,
                                   stratify=df_test['sentiment'].values)

df_train_full = pd.concat([df_train, df_val])

loading the NASDAQ dataset


In [ ]:
## loading the headline dataset for sentiment analysis


# BERT

In [25]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

bert_history, bert_test_outputs = fn.get_oof_and_test_preds(model_type='bert',
                                                            tokenizer=bert_tokenizer,
                                                            train_df=df_train_full,
                                                            test_df=df_test,
                                                            single_model=False,
                                                            loss_fn=loss_function)



Fold: 1


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


TypeError: unsupported operand type(s) for +: 'bool' and 'str'

In [ ]:
# load the bert model


# DistilBert


In [14]:

distilbert_tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
distilbert_history, distilbert_test_outputs = fn.get_oof_and_test_preds(model_type='distilbert',
                                                                     tokenizer=distilbert_tokenizer,
                                                                     train_df=df_train_full,
                                                                     test_df=df_test,
                                                                     single_model=False,
                                                                     loss_fn=loss_function)

fn.cv_ensemble_performance(distilbert_test_outputs, df_test['sentiment'].values)

Fold: 1
Epoch  1 / 5
--------------------------------------------------
Train Loss:  1.0005282180649893  |  Train Accuracy:  0.5464285714285714
Val Loss:  0.8288572090012687  |  Val Accuracy:  0.6307339449541285
Epoch Train Time:  0:00:30


Epoch  2 / 5
--------------------------------------------------
Train Loss:  0.677397473490968  |  Train Accuracy:  0.7206632653061225
Val Loss:  0.4950807967356273  |  Val Accuracy:  0.8096330275229358
Epoch Train Time:  0:00:29


Epoch  3 / 5
--------------------------------------------------
Train Loss:  0.42550665912579516  |  Train Accuracy:  0.8354591836734694
Val Loss:  0.3781126012493457  |  Val Accuracy:  0.8440366972477065
Epoch Train Time:  0:00:24


Epoch  4 / 5
--------------------------------------------------
Train Loss:  0.3112621467484503  |  Train Accuracy:  0.8821428571428571
Val Loss:  0.4146080070973507  |  Val Accuracy:  0.8302752293577982
Epoch Train Time:  0:00:21


Epoch  5 / 5
-----------------------------------------------

Train Loss:  0.21359342261694553  |  Train Accuracy:  0.924764090793165
Val Loss:  0.5143827156363321  |  Val Accuracy:  0.8436781609195402
Epoch Train Time:  0:00:24


Finished Training.
Fold Train Time:  0:02:06


Fold: 8
Epoch  1 / 5
--------------------------------------------------
Train Loss:  0.9938298284522886  |  Train Accuracy:  0.5705177250701352
Val Loss:  0.8511964476534298  |  Val Accuracy:  0.593103448275862
Epoch Train Time:  0:00:29


Epoch  2 / 5
--------------------------------------------------
Train Loss:  0.6785074904318747  |  Train Accuracy:  0.7077276205049732
Val Loss:  0.5248790319476809  |  Val Accuracy:  0.7724137931034483
Epoch Train Time:  0:00:31


Epoch  3 / 5
--------------------------------------------------
Train Loss:  0.40548141839785307  |  Train Accuracy:  0.846212700841622
Val Loss:  0.39549279239560875  |  Val Accuracy:  0.8344827586206897
Epoch Train Time:  0:00:27


Epoch  4 / 5
--------------------------------------------------
Train Loss:  

AttributeError: 'LabelEncoder' object has no attribute 'classes_'

In [ ]:
# load the DistilBert model

# RoBERT


In [17]:
roberta_tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
roberta_history, roberta_test_outputs = fn.get_oof_and_test_preds(model_type='roberta',
                                                               tokenizer=roberta_tokenizer,
                                                               train_df=df_train_full,
                                                               test_df=df_test,
                                                               single_model=False,
                                                               loss_fn=loss_function)

fn.cv_ensemble_performance(roberta_test_outputs, df_test['sentiment'].values)

Fold: 1
Epoch  1 / 5
--------------------------------------------------
Train Loss:  1.0370039596849558  |  Train Accuracy:  0.449234693877551
Val Loss:  0.8472018241882324  |  Val Accuracy:  0.5940366972477065
Epoch Train Time:  0:01:01


Epoch  2 / 5
--------------------------------------------------
Train Loss:  0.5754806747850106  |  Train Accuracy:  0.7525510204081632
Val Loss:  0.3507325291367514  |  Val Accuracy:  0.8440366972477065
Epoch Train Time:  0:03:00


Epoch  3 / 5
--------------------------------------------------
Train Loss:  0.31136169772671196  |  Train Accuracy:  0.8790816326530613
Val Loss:  0.41856292442285586  |  Val Accuracy:  0.8440366972477065
Epoch Train Time:  0:00:36


Epoch  4 / 5
--------------------------------------------------
Train Loss:  0.2543213488344027  |  Train Accuracy:  0.9071428571428571
Val Loss:  0.5553322678065992  |  Val Accuracy:  0.8371559633027523
Epoch Train Time:  0:00:35


Epoch  5 / 5
----------------------------------------------

Train Loss:  0.22910226555257218  |  Train Accuracy:  0.9196633511859219
Val Loss:  0.6083450123634455  |  Val Accuracy:  0.8551724137931035
Epoch Train Time:  0:00:42


Finished Training.
Fold Train Time:  0:04:10


Fold: 8
Epoch  1 / 5
--------------------------------------------------
Train Loss:  0.9977381835138894  |  Train Accuracy:  0.5934710533027289
Val Loss:  0.8704644877995763  |  Val Accuracy:  0.593103448275862
Epoch Train Time:  0:00:40


Epoch  2 / 5
--------------------------------------------------
Train Loss:  0.6375982224698958  |  Train Accuracy:  0.725580209130324
Val Loss:  0.37336670367845465  |  Val Accuracy:  0.8298850574712644
Epoch Train Time:  0:00:46


Epoch  3 / 5
--------------------------------------------------
Train Loss:  0.34196382969981287  |  Train Accuracy:  0.8648304004080591
Val Loss:  0.33836556658414857  |  Val Accuracy:  0.8344827586206897
Epoch Train Time:  0:00:40


Epoch  4 / 5
--------------------------------------------------
Train Loss:

AttributeError: 'LabelEncoder' object has no attribute 'classes_'

In [ ]:
# load the RoBERT model

# FinBERT


In [ ]:
sys.path.append(os.path.abspath("/FinBERT.py"))
import FinBERT


# LSTM


In [23]:
sys.path.append(os.path.abspath("/LSTM_pytorch.py"))
import LSTM_pytorch.LSTM as LSTM

In [2]:

filepath = f"./data/BTC_Data.csv"

hidden_dim = 20
layer_dim = 3  # ONLY CHANGE IS HERE FROM ONE LAYER TO TWO LAYER
output_dim = 1 #2
seq_dim = 7
correct = 0
total = 0
keys= ['High', 'Low', 'Open', 'Close', 'Volume', 'tweets',
       'Marketcap', 'google_trends', 'profitability', 'transaction_fee',
       'fluction'] 

targets=["fluction"]

model= LSTM(filepath=filepath,
           hidden_dim=hidden_dim,
           layer_dim=layer_dim,
           output_dim=output_dim,
           keys=keys,
           targets=targets)

NameError: name 'LSTM' is not defined

In [ ]:
# input_dim = len(df_scaled.columns)-1
# hidden_dim = 20
# layer_dim = 3  # ONLY CHANGE IS HERE FROM ONE LAYER TO TWO LAYER
# output_dim = 1

# model = LSTMModel(input_dim, hidden_dim, num_layers=layer_dim, output_dim=output_dim)


# criterion = nn.MSELoss()
# #TODO: also try L1Loss later
# #criterion= nn.L1Loss()

# learning_rate = 0.0001
# optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# seq_dim = 7

# iter = 0
# for epoch in range(num_epochs):
#     for features, labels in train_loader:
      
#         outputs = model(features)
#         labels=torch.tensor(labels,dtype=torch.float32)
#         # Calculate Loss: softmax --> cross entropy loss
#         loss = criterion(outputs, labels)
#         # Getting gradients w.r.t. parameters
#         loss.backward()
#         # Updating parameters
#         optimizer.step()
#         iter += 1

#         if iter % 500 == 0:
#             # Calculate Accuracy
#             correct = 0
#             total = 0
#             # Iterate through test dataset
#             for features, labels in test_loader:
#                 # Load features to a Torch Variable
#                 features = features.view(-1, seq_dim, input_dim).requires_grad_()

#                 # Forward pass only to get logits/output
#                 outputs = model(features)

#                 # Get predictions from the maximum value
#                 _, predicted = torch.max(outputs.data, 1)

#                 # Total number of labels
#                 total += labels.size(0)

#                 # Total correct predictions
#                 correct += (predicted == labels).sum()

#             accuracy = 100 * correct / total

#             # Print Loss
#             print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss, accuracy))

# print("finished")


# torch.save(model.state_dict(), './model/LSTM_state_dict_'+datetime.datetime.today().strftime('%Y_%m_%d')+'.pt')